In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import os

from transformers import pipeline

import requests
import json
import openai

from medisure.ml_logic.data import clean_text, eda_pipeline
from medisure.ml_logic.FDI import get_tau_score
from medisure.ml_logic.DDI import ddi_data

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/raynasser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/raynasser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/raynasser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
fin_medi = pd.read_csv('/Users/raynasser/code/raynasser/MediSure/data/raw_data/final_medi.csv')
fin_medi

,Unnamed: 0,id,name,substitute0,substitute1,substitute2,substitute3,substitute4,sideEffect0,sideEffect1,...,use2,use3,use4,Chemical Class,Habit Forming,Therapeutic Class,Action Class,name_,dosage,form
0,0,1,augmentin 625 duo tablet,Penciclav 500 mg/125 mg Tablet,Moxikind-CV 625 Tablet,Moxiforce-CV 625 Tablet,Fightox 625 Tablet,Novamox CV 625mg Tablet,Vomiting,Nausea,...,Not specified,Not specified,Not specified,Other,No,ANTI INFECTIVES,NaN,augmentin,625,duo tablet
1,1,2,azithral 500 tablet,Zithrocare 500mg Tablet,Azax 500 Tablet,Zady 500 Tablet,Cazithro 500mg Tablet,Trulimax 500mg Tablet,Vomiting,Nausea,...,Not specified,Not specified,Not specified,Macrolides,No,ANTI INFECTIVES,Macrolides,azithral,500,tablet
2,2,3,ascoril ls syrup,Solvin LS Syrup,Ambrodil-LX Syrup,Zerotuss XP Syrup,Capex LS Syrup,Broxum LS Syrup,Nausea,Vomiting,...,Not specified,Not specified,Not specified,Other,No,RESPIRATORY,NaN,ascoril ls syrup,NaN,NaN
3,3,5,avil 25 tablet,Eralet 25mg Tablet,No substitute,No substitute,No substitute,No substitute,Sleepiness,Dryness in mouth,...,Not specified,Not specified,Not specified,Pyridines Derivatives,No,RESPIRATORY,H1 Antihistaminics (First Generation),avil,25,tablet
4,4,6,allegra-m tablet,Emlukast-FX Tablet,LCFEX-Mont Tablet,Fixar 10mg/120mg Tablet,Histakind-M Tablet,Histafree-M Tablet,Nausea,Diarrhea,...,Not specified,Not specified,Not specified,Other,No,RESPIRATORY,NaN,allegra-m tablet,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222820,222820,248214,zestrain 100mg/325mg/250mg tablet mr,Aclotec MR 100mg/325mg/250mg Tablet,SAMONEC MR TABLET,Uniclof 100mg/325mg/250mg Tablet MR,Macito 100mg/325mg/250mg Tablet MR,Rumatin 100mg/325mg/250mg Tablet MR,Nausea,Vomiting,...,Not specified,Not specified,Not specified,Other,No,PAIN ANALGESICS,NaN,zestrain,100mg/325mg/250mg,tablet mr
222821,222821,248215,zoxinace 200mg tablet sr,Algic 200mg Tablet SR,Topnac 200mg Tablet SR,Kindac 200mg Tablet SR,Ultranac 200mg Tablet SR,Bignac 200mg Tablet SR,Dizziness,Indigestion,...,Not specified,Not specified,Not specified,Dichlorobenzenes Derivative,No,PAIN ANALGESICS,NSAID's- Non-Selective COX 1&2 Inhibitors (ace...,zoxinace,200mg,tablet sr
222822,222822,248216,zivex 25mg tablet,HD Zine 25mg Tablet,Hydrocas 25mg Tablet,Hyzox 25 Tablet,Hydil 25mg Tablet,Zyzine 25mg Tablet,Sedation,Nausea,...,Not specified,Not specified,Not specified,Piperazine Derivative,No,RESPIRATORY,H1 Antihistaminics (First Generation),zivex,25mg,tablet
222823,222823,248217,zi fast 500mg injection,Zycin 500mg Injection,Aziwok 500mg Injection,Azirow 500mg Injection,Toracin 500mg Injection,Azymed 500mg Injection,"Injection site reactions (pain, swelling, redn...",Vomiting,...,Not specified,Not specified,Not specified,Macrolides,No,ANTI INFECTIVES,Macrolides,zi fast,500mg,injection


In [4]:
med_unq_drug = pd.DataFrame(fin_medi.name_.unique(), columns= ['drug'])
med_unq_drug.to_csv('/Users/raynasser/code/raynasser/MediSure/data/raw_data/uniq_medicine.csv')

In [5]:
med_unq_drugs = pd.read_csv('/Users/raynasser/code/raynasser/MediSure/data/raw_data/uniq_medicine.csv')
med_unq_drugs.drop(columns='Unnamed: 0', inplace=True)

In [6]:
def lower(text):
    try:
        return text.lower()
    except:
        return text




med_unq_drugs.applymap(lower)

,drug
0,augmentin
1,azithral
2,ascoril ls syrup
3,avil
4,allegra-m tablet
...,...
171966,zeotaz
171967,zef cv
171968,zestrain
171969,zoxinace


In [7]:
med_unq_drugs.duplicated().sum()

0

In [8]:
from medisure.ml_logic.InChi import uniq_drugs_key, fetch_chemical_info

# med_unq_drugs = med_unq_drugs.head(7)

med_unq_drugs[['SMILES', 'InChI', 'InChIKey']] = med_unq_drugs['drug'].apply(lambda x: pd.Series(fetch_chemical_info(x)))


Error: Unable to fetch data for azithral. Status code: 404
Error: Unable to fetch data for ascoril ls syrup. Status code: 404
Error: Unable to fetch data for avil. Status code: 404
Error: Unable to fetch data for allegra-m tablet. Status code: 404
Error: Unable to fetch data for ascoril d plus syrup sugar free. Status code: 404
Error: Unable to fetch data for alex syrup. Status code: 404
Error: Unable to fetch data for anovate cream. Status code: 404
Error: Unable to fetch data for augmentin duo oral suspension. Status code: 404
Error: Unable to fetch data for ambrodil-s syrup. Status code: 404
Error: Unable to fetch data for arkamin tablet. Status code: 404
Error: Unable to fetch data for avomine tablet. Status code: 404
Error: Unable to fetch data for asthalin syrup. Status code: 404
Error: Unable to fetch data for altraday capsule sr. Status code: 404
Error: Unable to fetch data for ascoril ls junior syrup. Status code: 404
Error: Unable to fetch data for aciloc rd. Status code: 404

KeyboardInterrupt: 

In [13]:
med_unq_drugs

,drug,SMILES,InChI,InChIKey
0,augmentin,CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)[C@@H]...,InChI=1S/C16H19N3O5S.C8H9NO5.K/c1-16(2)11(15(2...,DWHGNUUWCJZQHO-ZVDZYBSKSA-M
1,azithral,Not Found,Not Found,Not Found
2,ascoril ls syrup,Not Found,Not Found,Not Found
3,avil,Not Found,Not Found,Not Found
4,allegra-m tablet,Not Found,Not Found,Not Found
5,amoxyclav,CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)[C@@H]...,InChI=1S/C16H19N3O5S.C8H9NO5/c1-16(2)11(15(23)...,QJVHTELASVOWBE-AGNWQMPPSA-N
6,atarax,C1CN(CCN1CCOCCO)C(C2=CC=CC=C2)C3=CC=C(C=C3)Cl,InChI=1S/C21H27ClN2O2/c22-20-8-6-19(7-9-20)21(...,ZQDWXGKKHFNSQK-UHFFFAOYSA-N


In [36]:
med_unq_drugs7 = med_unq_drugs.iloc[60000 : 70000]
med_unq_drugs8 = med_unq_drugs.iloc[70000 : 80000]
med_unq_drugs9 = med_unq_drugs.iloc[80000 : 90000]
med_unq_drugs10 = med_unq_drugs.iloc[90000 : 100000]
med_unq_drugs11 = med_unq_drugs.iloc[100000 : 110000]
med_unq_drugs12 = med_unq_drugs.iloc[110000 : 120000]
med_unq_drugs13 = med_unq_drugs.iloc[120000 : 130000]
med_unq_drugs14 = med_unq_drugs.iloc[130000 : 140000]
med_unq_drugs15 = med_unq_drugs.iloc[140000 : 150000]
med_unq_drugs16 = med_unq_drugs.iloc[150000 : 160000]
med_unq_drugs17 = med_unq_drugs.iloc[160000 : 170000]
med_unq_drugs18 = med_unq_drugs.iloc[170000 : 180000]

med_unq_drugs18

,drug
170000,zoreo
170001,zithrel
170002,zodal
170003,zentro-ls capsule sr
170004,zfen-sp tablet
...,...
171966,zeotaz
171967,zef cv
171968,zestrain
171969,zoxinace


In [37]:
from medisure.ml_logic.InChi import uniq_drugs_key, fetch_chemical_info



med_unq_drugs13[['SMILES', 'InChI', 'InChIKey']] = med_unq_drugs13['drug'].apply(lambda x: pd.Series(fetch_chemical_info(x)))


Error: Unable to fetch data for oxizim. Status code: 503
Error: Unable to fetch data for oflaquin. Status code: 503
Error: Unable to fetch data for ocelv-m tablet. Status code: 503
Error: Unable to fetch data for omper. Status code: 503
Error: Unable to fetch data for oriva cms. Status code: 503
Error: Unable to fetch data for optolin eye drop. Status code: 503
Error: Unable to fetch data for omehelp. Status code: 503
Error: Unable to fetch data for osteomap tablet. Status code: 503
Error: Unable to fetch data for ostorin. Status code: 503
Error: Unable to fetch data for omehelp-d. Status code: 503
Error: Unable to fetch data for oflosa o tablet. Status code: 503
Error: Unable to fetch data for ofil eye. Status code: 503
Error: Unable to fetch data for ocuglow eye drop. Status code: 503
Error: Unable to fetch data for omepal. Status code: 503
Error: Unable to fetch data for odol mr. Status code: 503
Error: Unable to fetch data for omicad. Status code: 503
Error: Unable to fetch data fo

/var/folders/qx/yrwdfm5j0jz1gmhp0x2tvjy80000gn/T/ipykernel_92022/3602757555.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  med_unq_drugs13[['SMILES', 'InChI', 'InChIKey']] = med_unq_drugs13['drug'].apply(lambda x: pd.Series(fetch_chemical_info(x)))
/var/folders/qx/yrwdfm5j0jz1gmhp0x2tvjy80000gn/T/ipykernel_92022/3602757555.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  med_unq_drugs13[['SMILES', 'InChI', 'InChIKey']] = med_unq_drugs13['drug'].apply(lambda x: pd.Series(fetch_chemical_info(x)))
/var

In [38]:
med_unq_drugs13

,drug,SMILES,InChI,InChIKey
120000,oxizim,Not Found,Not Found,Not Found
120001,oflaquin,Not Found,Not Found,Not Found
120002,ocelv-m tablet,Not Found,Not Found,Not Found
120003,omper,Not Found,Not Found,Not Found
120004,oriva cms,Not Found,Not Found,Not Found
...,...,...,...,...
129995,pantag-d,Not Found,Not Found,Not Found
129996,pegasys,Not Found,Not Found,Not Found
129997,perigesic-mr tablet,Not Found,Not Found,Not Found
129998,phardep,Not Found,Not Found,Not Found


In [33]:
med_unq_drugs13.to_csv('/Users/raynasser/code/raynasser/MediSure/data/raw_data/med_unq_drugs/13.csv')

In [39]:
med_unq_drugs13[med_unq_drugs13.InChIKey == 'Not Found']

,drug,SMILES,InChI,InChIKey
120000,oxizim,Not Found,Not Found,Not Found
120001,oflaquin,Not Found,Not Found,Not Found
120002,ocelv-m tablet,Not Found,Not Found,Not Found
120003,omper,Not Found,Not Found,Not Found
120004,oriva cms,Not Found,Not Found,Not Found
...,...,...,...,...
129995,pantag-d,Not Found,Not Found,Not Found
129996,pegasys,Not Found,Not Found,Not Found
129997,perigesic-mr tablet,Not Found,Not Found,Not Found
129998,phardep,Not Found,Not Found,Not Found


In [35]:
med_unq_drugs13 = med_unq_drugs13[med_unq_drugs13.InChIKey != 'Not Found']

med_unq_drugs13

,drug,SMILES,InChI,InChIKey


In [31]:
med_unq_drugs13.to_csv('/Users/raynasser/code/raynasser/MediSure/data/raw_data/med_unq_drugs/13_.csv')

In [65]:
fin_prof = pd.read_csv('/Users/raynasser/code/raynasser/MediSure/data/raw_data/profiles/fin_profiles.csv')
fin_prof.columns

Index(['Unnamed: 0', 'PATIENT', 'FIRST', 'MIDDLE', 'LAST', 'GENDER', 'ADDRESS',
       'CITY', 'Age', 'Allergy 1', 'Allergy 2', 'Allergy 3', 'Allergy 4',
       'Allergy 5', 'Medication 1', 'Medication 2', 'Medication 3',
       'Medication 4', 'Medication 5', 'Medication 6', 'Medication 1 Name',
       'Medication 2 Name', 'Medication 3 Name', 'Medication 4 Name',
       'Medication 5 Name', 'Medication 6 Name'],
      dtype='object')

In [66]:
fin_prof.shape


(2267, 26)

In [44]:
keys = pd.read_csv('/Users/raynasser/code/raynasser/MediSure/data/raw_data/profiles/prof_drug_key.csv')
keys.drop(columns='Unnamed: 0', inplace=True)

In [46]:
keys.columns

Index(['drug', 'InChIKey'], dtype='object')

In [47]:
# fin_prof = fin_prof.head(7)

In [67]:
for i in range(1, 7):
    medication_col = f'Medication {i} Name'
    inchi_col = f'Medication {i} InChIKey'


    fin_prof = fin_prof.merge(
        keys, how='left', left_on=medication_col, right_on='drug'
    ).rename(columns={'InChIKey': inchi_col}).drop(columns=['drug'])

    fin_prof[inchi_col] = fin_prof[inchi_col].fillna('Unknown')


In [71]:
fin_prof.to_csv('/Users/raynasser/code/raynasser/MediSure/data/raw_data/profiles/fin_profiles_keys.csv')

In [70]:
fin_prof.isna().sum()

Unnamed: 0                 0
PATIENT                    0
FIRST                      0
MIDDLE                   438
LAST                       0
GENDER                     0
ADDRESS                    0
CITY                       0
Age                        0
Allergy 1                  0
Allergy 2                  0
Allergy 3                  0
Allergy 4                  0
Allergy 5                  0
Medication 1               0
Medication 2               0
Medication 3               0
Medication 4               0
Medication 5               0
Medication 6               0
Medication 1 Name          0
Medication 2 Name          0
Medication 3 Name          0
Medication 4 Name          0
Medication 5 Name          0
Medication 6 Name          0
Medication 1 InChIKey      0
Medication 2 InChIKey      0
Medication 3 InChIKey      0
Medication 4 InChIKey      0
Medication 5 InChIKey      0
Medication 6 InChIKey      0
dtype: int64

In [73]:
fdi = pd.read_csv('/Users/raynasser/code/raynasser/MediSure/data/raw_data/FooDru/fdi_tau_results-3.csv')
fdi

,texts_ID,food,drug,clean_text,summarize,food_,drug_,relationship,tau_score
0,0,grapefruit,abemaciclib,Moderate Food Interaction||GENERALLY AVOID: G...,A high-calorie meal may increase the concentr...,high-calorie meal,abemaciclib,improving health,0.50
1,2,grapefruit,fentanyl,GENERALLY AVOID: Consumption of grapefruit ju...,The World Health Organization (WHO) warns tha...,grapefruit juice,fentanyl,degrading health,-0.70
2,7,grapefruit,paclitaxel,Moderate Food Interaction||MONITOR: Coadminis...,"AUC (AUC) for docetaxel, a drug used to treat...",grapefruit juice,docetaxel,degrading health,-0.65
3,11,ethanol,opioid analgesics,Moderate Drug Interaction||GENERALLY AVOID: E...,Ethanol can be used as a treatment for painki...,Ethanol,Painkillers,improving health,0.70
4,13,ethanol,dextromethorphan,Moderate Drug Interaction||GENERALLY AVOID: T...,Dextromethorphan and ethanol can be used as a ...,ethanol,dextromethorphan,improving health,0.70
...,...,...,...,...,...,...,...,...,...
8244,8441,insulin,d-glucose,"Indeed, the many capillary blood glucose measu...",The use of an IV insulin pump to monitor bloo...,NaN,IV insulin,improving health,0.80
8245,8442,sucrose,d-glucose,Glycemic Optimization On Discharge From the Em...,A new study has shown that using a continuous...,NaN,continuous glucose monitor,improving health,0.70
8246,8443,amino acid,phenylalanine,PANDA: PKU Amino Acid Evaluation Phenylketonur...,A rare genetic disorder called Phenylketonuri...,phenylalanine,None,degrading health,-1.00
8247,8444,diet,phenylalanine,SNAP: Study Nutrients in Adult PKU Phenylketon...,Researchers at the University of Aberdeen are ...,NaN,NaN,neutral,0.00


In [77]:
fdi_key = pd.read_csv('/Users/raynasser/code/raynasser/MediSure/data/raw_data/FooDru/unique_foods_key.csv')
fdi_key.drop(columns='Unnamed: 0', inplace=True)

In [78]:
fdi_key.columns

Index(['drug', 'SMILES', 'InChI', 'InChIKey'], dtype='object')

In [79]:
fdi.columns

Index(['texts_ID', 'food', 'drug', 'clean_text', 'summarize', 'food_', 'drug_',
       'relationship', 'tau_score'],
      dtype='object')

In [80]:


fin_fdi = pd.merge(fdi, fdi_key[['drug', 'InChIKey']], on='drug', how='left')
fin_fdi

# fin_fdi[inchi_col] = fin_fdi[inchi_col].fillna('Unknown')

,texts_ID,food,drug,clean_text,summarize,food_,drug_,relationship,tau_score,InChIKey
0,0,grapefruit,abemaciclib,Moderate Food Interaction||GENERALLY AVOID: G...,A high-calorie meal may increase the concentr...,high-calorie meal,abemaciclib,improving health,0.50,UZWDCWONPYILKI-UHFFFAOYSA-N
1,2,grapefruit,fentanyl,GENERALLY AVOID: Consumption of grapefruit ju...,The World Health Organization (WHO) warns tha...,grapefruit juice,fentanyl,degrading health,-0.70,PJMPHNIQZUBGLI-UHFFFAOYSA-N
2,7,grapefruit,paclitaxel,Moderate Food Interaction||MONITOR: Coadminis...,"AUC (AUC) for docetaxel, a drug used to treat...",grapefruit juice,docetaxel,degrading health,-0.65,RCINICONZNJXQF-MZXODVADSA-N
3,11,ethanol,opioid analgesics,Moderate Drug Interaction||GENERALLY AVOID: E...,Ethanol can be used as a treatment for painki...,Ethanol,Painkillers,improving health,0.70,BWFZGLIDGVJKML-ACOWMKLESA-N
4,13,ethanol,dextromethorphan,Moderate Drug Interaction||GENERALLY AVOID: T...,Dextromethorphan and ethanol can be used as a ...,ethanol,dextromethorphan,improving health,0.70,MKXZASYAUGDDCJ-NJAFHUGGSA-N
...,...,...,...,...,...,...,...,...,...,...
8244,8441,insulin,d-glucose,"Indeed, the many capillary blood glucose measu...",The use of an IV insulin pump to monitor bloo...,NaN,IV insulin,improving health,0.80,NaN
8245,8442,sucrose,d-glucose,Glycemic Optimization On Discharge From the Em...,A new study has shown that using a continuous...,NaN,continuous glucose monitor,improving health,0.70,NaN
8246,8443,amino acid,phenylalanine,PANDA: PKU Amino Acid Evaluation Phenylketonur...,A rare genetic disorder called Phenylketonuri...,phenylalanine,None,degrading health,-1.00,COLNVLDHVKWLRT-QMMMGPOBSA-N
8247,8444,diet,phenylalanine,SNAP: Study Nutrients in Adult PKU Phenylketon...,Researchers at the University of Aberdeen are ...,NaN,NaN,neutral,0.00,COLNVLDHVKWLRT-QMMMGPOBSA-N


In [88]:
fin_fdi.isna().sum()

texts_ID           0
food               0
drug               0
clean_text         0
summarize          0
food_           5129
drug_           2101
relationship     443
tau_score          0
InChIKey        4670
dtype: int64

In [92]:
fin_fdi.to_csv('/Users/raynasser/code/raynasser/MediSure/data/raw_data/FooDru/fin_fdi_key.csv')

In [82]:

base_url = "https://cactus.nci.nih.gov/chemical/structure/{}/inchikey"


def fetch_chemical_info(drug_name):
    try:

        drug_encoded = drug_name.replace(' ', '%20')


        response = requests.get(base_url.format(drug_encoded))

        if response.status_code == 200:
            inchi_key = response.text.strip().replace('InChIKey=', '')
            return pd.Series([None, None, inchi_key])
        else:
            return pd.Series([None, None, None])

    except Exception as e:
        print(f"{drug_name}: Error - {e}")
        return pd.Series([None, None, None])

In [86]:
fin_fdi['InChIKey_'] = fin_fdi['drug_'].apply(lambda x: pd.Series(fetch_chemical_info(x)))


nan: Error - 'float' object has no attribute 'replace'
nan: Error - 'float' object has no attribute 'replace'
nan: Error - 'float' object has no attribute 'replace'
nan: Error - 'float' object has no attribute 'replace'
Budesonide: Error - ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
ibrexafungerp: Error - ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
encorafenib: Error - ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Buspirone: Error - ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
levodopa: Error - ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
cannabidiol: Error - ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
rilpivirine: Error - ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Cabotegravir: Error - ('Connection aborted.', ConnectionResetError(54, 'Connection reset by

KeyboardInterrupt: 

In [85]:
fin_fdi.shape

(8249, 10)